In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime
import time
import sklearn.preprocessing as preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sys import path
path.append(r'E:\data_mining\project\IJCAI-17-口碑\koubei')

import predict
import base
import shop_classify as sc

import matplotlib
myfont = matplotlib.font_manager.FontProperties(fname=r'C:/Windows/Fonts/msyh.ttf')
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False 

%load_ext autoreload
%autoreload 2

C:\Program Files\Anaconda3\lib\site-packages\matplotlib\__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
#读取休假表
calenders = base.ReadCalendarCSV();
#读取user_pay_count
user_pay_counts = base.ReadUserPayCountCSV();

##读取城市名称对照表
city_names = base.ReadCityNameCSV();
#读取开店时间数据
shop_open_dates = base.ReadShopOpenDateCSV();

In [3]:
###读取商家信息
shop_infos = base.ReadProcessShopInfoCSV();

In [4]:
%reload_ext autoreload

In [5]:
def initWeekDate(start_time, stop_time):
    train_group_time_ranges = []
    while(start_time <= stop_time):
        X_end_time = start_time + datetime.timedelta(weeks=2, days=6)
        y_start_time = start_time + datetime.timedelta(weeks=3)
        y_end_time = y_start_time + datetime.timedelta(days=6)
        
        train_group_time_ranges.append([pd.date_range(start=start_time, end=X_end_time, freq='D'), pd.date_range(start=y_start_time, end=y_end_time, freq='D')])
        
        start_time = start_time + datetime.timedelta(weeks=1)

    return train_group_time_ranges

In [24]:
start_time = pd.to_datetime('2016-6-7')
stop_time = pd.to_datetime('2016-10-31') - datetime.timedelta(weeks=3, days=6)
#stop_time = pd.to_datetime('2016-10-31') - datetime.timedelta(weeks=3, days=6)
time_ranges = initWeekDate(start_time, stop_time)

In [7]:
train_pay_counts = pd.read_csv("../data/train/fill_sliding_window_data.txt", sep="\t", encoding='UTF-8', index_col=0)

In [8]:
train_pay_counts.columns = train_pay_counts.columns.map(lambda str_date:datetime.datetime.strptime(str_date, "%Y-%m-%d"))

In [9]:
def calculateSingleHolidayMeanData(id, shop_open_dates, train_pay_counts, calenders, week):
    df = train_pay_counts.loc[id][np.array(range(week * 7, (week + 1) * 7))].T;
    df = df.to_frame();
    df.columns = ['count']
    df['holiday'] = calenders.loc[df.index.strftime('%Y-%m-%d')]['daytype'].values
        
    params = {}
    
    params['w_' + str(week) + '_mean'] = df['count'].mean()
    params['w_' + str(week) + '_std'] = df['count'].std()
    params['w_' + str(week) + '_max'] = df['count'].max()
    params['w_' + str(week) + '_min'] = df['count'].min()
    params['w_' + str(week) + '_median'] = df['count'].median()
    params['w_' + str(week) + '_mad'] = df['count'].mad()
    
    for i in range(1, 6):
            temp_df = df[df['holiday'] == i];
            if len(temp_df) == 0:
                params['w_' + str(week) + '_daytype_mean_' + str(i)] = params['w_' + str(week) + '_mean']
            else:
                params['w_' + str(week) + '_daytype_mean_' + str(i)] = temp_df['count'].mean()
            
    return params;

In [10]:
max_shop_num = 2001

In [33]:
def calculateHolidayMeansData(shop_open_dates, train_pay_counts, calenders):
    shop_param_dict = {}
    for id in range(1,max_shop_num):
        param_dict = {}
        for week in range(3):
            params = calculateSingleHolidayMeanData(id, shop_open_dates, train_pay_counts, calenders, week)
            param_dict.update(params)

        ######################################################################################
        all_week_counts = train_pay_counts.loc[id][np.array(range(0, 21))].T.to_frame();
        all_week_counts.columns = ['count']
        #print(all_week_counts)
        all_week_counts['holiday'] = calenders.loc[all_week_counts.index.strftime('%Y-%m-%d')]['daytype'].values
        
        params = {}
        params['all_mean'] = all_week_counts['count'].mean()
        params['all_std'] = all_week_counts['count'].std()
        params['all_max'] = all_week_counts['count'].max()
        params['all_min'] = all_week_counts['count'].min()
        params['all_median'] = all_week_counts['count'].median()
        params['all_mad'] = all_week_counts['count'].mad()
    
        for i in range(1, 6):
            temp_df = all_week_counts[all_week_counts['holiday'] == i];
            if len(temp_df) == 0:
                params['all_daytype_mean_' + str(i)] = params['all_mean']
                params['all_daytype_all_max_' + str(i)] = params['all_max']
                params['all_daytype_all_min_' + str(i)] = params['all_min']
                params['all_daytype_all_median_' + str(i)] = params['all_median']
                params['all_daytype_all_mad_' + str(i)] = params['all_mad']
            else:
                params['all_daytype_mean_' + str(i)] = temp_df['count'].mean()
                params['all_daytype_max_' + str(i)] = temp_df['count'].max()
                params['all_daytype_min_' + str(i)] = temp_df['count'].min()
                params['all_daytype_median_' + str(i)] = temp_df['count'].median()
                params['all_daytype_mad_' + str(i)] = temp_df['count'].mad()
                
            if len(temp_df) <= 1:
                params['all_daytype_all_std_' + str(i)] = params['all_std']
            else:
                params['all_daytype_std_' + str(i)] = temp_df['count'].std()
                
        param_dict.update(params)
                
        shop_param_dict[id] = param_dict;
    return shop_param_dict

In [34]:
all_X_train_df = None
all_y_train_df = None
is_first_batch = True

for tr in time_ranges:
    X_train_data = train_pay_counts[tr[0]]
    y_train_data = train_pay_counts[tr[1]]
    print(tr[0])
    print('shop_mean_dict')
    shop_mean_dict = calculateHolidayMeansData(shop_open_dates, X_train_data, calenders)
    ########################################################################################
    X_train_df = pd.DataFrame(shop_mean_dict)
    X_train_df = X_train_df.T
    #前三周数据
    ############################################################################################
    X_train_df = pd.merge(X_train_df, X_train_data, left_index=True, right_index=True)
    
    ############################################################################################
    
    #poly = PolynomialFeatures(2, interaction_only=True, include_bias=False)
    #X_train_df = pd.merge(X_train_df, shop_infos[['score', 'shop_level', 'per_pay']], how='inner', left_on='id', right_index=True)

    #X_train_id_df = X_train_df['id']
    #X_train_df = X_train_df.drop('id', axis=1)

    #X_train_poly = poly.fit_transform(X_train_df.values)
    X_train_scaled = preprocessing.scale(X_train_df)
    X_train_df = pd.DataFrame(X_train_scaled)

    #X_train_df['id'] = X_train_id_df

    #X_train_df = pd.merge(X_train_df, shop_infos.drop(['score', 'shop_level', 'per_pay'], axis=1), how='inner', left_on='id',right_index=True)
    #X_train_w1_df = X_train_w1_df.astype(np.int)
    #X_train_df = X_train_df.drop(['id'], axis=1)
    
    
    y_train_data.columns=np.array(range(7))
    y_train_df = y_train_data
    
    
    if is_first_batch == True:
        all_X_train_df = X_train_df
        all_y_train_df = y_train_df
        is_first_batch = False
    else:
        #print(all_X_train_df)
        all_X_train_df = pd.concat([all_X_train_df, X_train_df])
        all_y_train_df = pd.concat([all_y_train_df, y_train_df])


DatetimeIndex(['2016-06-07', '2016-06-08', '2016-06-09', '2016-06-10',
               '2016-06-11', '2016-06-12', '2016-06-13', '2016-06-14',
               '2016-06-15', '2016-06-16', '2016-06-17', '2016-06-18',
               '2016-06-19', '2016-06-20', '2016-06-21', '2016-06-22',
               '2016-06-23', '2016-06-24', '2016-06-25', '2016-06-26',
               '2016-06-27'],
              dtype='datetime64[ns]', freq='D')
shop_mean_dict
DatetimeIndex(['2016-06-14', '2016-06-15', '2016-06-16', '2016-06-17',
               '2016-06-18', '2016-06-19', '2016-06-20', '2016-06-21',
               '2016-06-22', '2016-06-23', '2016-06-24', '2016-06-25',
               '2016-06-26', '2016-06-27', '2016-06-28', '2016-06-29',
               '2016-06-30', '2016-07-01', '2016-07-02', '2016-07-03',
               '2016-07-04'],
              dtype='datetime64[ns]', freq='D')
shop_mean_dict
DatetimeIndex(['2016-06-21', '2016-06-22', '2016-06-23', '2016-06-24',
               '2016-06-25', '201

In [35]:
all_X_train_df.to_csv("../data/train/gbdt/X_train.txt", sep='\t', index=True, encoding='UTF-8')
all_y_train_df.to_csv("../data/train/gbdt/y_train.txt", sep='\t', header= None, index=True, encoding='UTF-8')

In [42]:
all_X_train_df.columns

RangeIndex(start=0, stop=26, step=1)